# Immoscout, large

A more sophisticated approach with JQueries on those pages. Then let's get the artillery.

In [10]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait # let it wait for some time. patience while loading it all
import time
from selenium.webdriver.common.by import By # search for by param
from selenium.webdriver.common.keys import Keys # we want to press enter at some stage
from selenium.webdriver.support import expected_conditions as EC # what am i looking for, has the page loaded?
from selenium.common.exceptions import TimeoutException # handle timeouts
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import multiprocessing as mp
from selenium.webdriver.firefox.options import Options
import pickle
import gc
import os

options = Options()
options.set_headless(True) # newer webdriver versions

cities = ['berlin', 'hamburg', 'muenchen', "stuttgart", "duesseldorf","koeln", "frankfurt"]
driver = webdriver.Chrome(options = options)
area_cluster = {}
print('scraping cities')
for city in cities:

    driver.get(f'https://www.immobilienscout24.de/neubau/{city}.html')
    time.sleep(5)
    regions = driver.find_elements_by_css_selector("div.section.region--section.grid.gutter.margin-bottom-l")
    results_raw = [tag for tag in regions]
    bezirke = [result.find_element_by_tag_name('h3').text for result in results_raw]
    for index,bezirk in enumerate(bezirke):
        area_cluster[bezirk] = [results_raw[index],index,city]
        
    caption = [result.find_elements_by_css_selector('h3.font-m.font-semibold.font-ellipsis') for result in results_raw]
    flat_caption = [item for sublist in caption for item in sublist if isinstance(sublist,list)]
    caption = [e.text for e in flat_caption]
    
    price = [result.find_elements_by_css_selector('span.absolute-content.padding-vertical-xs.padding-horizontal.font-ellipsis.font-right.font-m.font-semibold.font-white.projectresultlist__price') for result in results_raw]
    flat_price = [item for sublist in price for item in sublist if isinstance(sublist,list)]
    price = [e.text for e in flat_price]
    
    specs = [result.find_elements_by_css_selector('div.grid-item.one-half.font-right.font-ellipsis') for result in results_raw]
    flat_specs = [item for sublist in specs for item in sublist if isinstance(sublist,list)]
    
    area = [e.text for e in flat_specs[0::3]]
    units = [e.text for e in flat_specs[1::3]]
    availability = [e.text for e in flat_specs[2::3]]
    
    href = [result.find_elements_by_css_selector('a') for result in results_raw]
    flat_href = [item for sublist in href for item in sublist if isinstance(sublist,list)]
    href = [e.get_attribute('href') for e in flat_href]
    
    if city == 'berlin':
        df = pd.DataFrame(data = [caption,price,area,units,availability,href]).transpose()
        df['city'] = city
    else:
        df2 = pd.DataFrame(data = [caption,price,area,units,availability,href]).transpose()
        df2['city'] = city
        df = df.append(df2)
    os.system('pkill firefox')
    os.system('pkill chrome')
    os.system('pkill chromium')

df.columns = ['name', 'price', 'area', 'units', 'availability', 'link', 'city']
df['price_min'] = df['price'].apply(lambda x: re.search(r'(.*) - (.*) €',x)[1].replace('.',''))
df['price_max'] = df['price'].apply(lambda x: re.search(r'(.*) - (.*) €',x)[2].replace('.',''))
df.reset_index(inplace = True)
pickle.dump(df, open('cities_df','wb'))

links = df['link'].tolist()

project_names = []
fineprints = []
texts = []
images_set = []
promoters = []
promoter_hp = []
contact_names = []
phone_contact = []

for index, link in enumerate(links):
    progress = index / len(links)
    clear_output(wait=True)
    print(f'scraping single projects, progress is {progress}')
    driver = webdriver.Chrome(options=options)

    driver.get(link)
    try:
        driver.find_element_by_id('consent_prompt_submit').click()
    except Exception as e:
        continue
    try:
        name = driver.find_elements_by_tag_name("h1")
        name = [e.text for e in name][0]
        project_names.append(name)
    except Exception as e:
        project_names.append('')
        continue
    try:
        driver.find_element_by_css_selector('a.icon-arrow.margin-top-xs').click()
    except:
        continue
    try:
        fineprint = driver.find_elements_by_css_selector("p.fineprint")
        fineprint = [e.text for e in fineprint]
        fineprints.append(fineprint)
    except:
        fineprints.append('')
        continue
    try:
        text = driver.find_elements_by_tag_name('p')
        text= [e.text for e in text if len(e.text) > 90]
        texts.append(''.join(text))
    except:
        texts.append('')
        continue
    try:    
        images = driver.find_elements_by_tag_name('img')
        images = [e.get_attribute('src') for e in images]
        images = [str(e) for e in images if 'amazonaws' in str(e)]
        images_set.append(images)
    except:
        images.append('')
        continue
    try:
        promoters.append(driver.find_element_by_css_selector('a.homepage-url').text)
        promoter_hp.append(driver.find_element_by_css_selector('a.homepage-url').get_attribute('href'))
        contact_names.append(driver.find_element_by_id('detailsContactFooter').find_elements_by_tag_name('p')[0].text)
    except:
        promoters.append('')
        promoter_hp.append('')
        contact_names.append('')
        continue
    driver.close()
    driver.quit()
    print(fineprint)
    gc.collect()
    os.system('pkill firefox')
    os.system('pkill chrome')
    os.system('pkill chromium')
    os.system('pkill nightly')
df['project_names'] = project_names
df['fineprints'] = fineprints
df['texts'] = texts
df['images_set'] = images_set
df['promoters'] = promoters
df['promoter_hp'] = promoter_hp
df['contact_names'] = contact_names
df['phone_contact'] = phone_contact
df.to_csv('bum.csv', encoding = 'utf-8')

scraping single projects, progress is 0.9980392156862745


ValueError: Length of values does not match length of index

In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait # let it wait for some time. patience while loading it all
import time
from selenium.webdriver.common.by import By # search for by param
from selenium.webdriver.support import expected_conditions as EC # what am i looking for, has the page loaded?
from selenium.common.exceptions import TimeoutException # handle timeouts
from selenium.webdriver.firefox.options import Options
import re
from IPython.display import clear_output
from selenium.webdriver.firefox.options import Options
import gc
import os
from datetime import datetime
import pickle

now = str(datetime.now()).replace(':','_').replace(' ','_').replace('-','_')

options = webdriver.ChromeOptions()
options.add_argument('headless') # newer webdriver versions

cities = ['berlin', 'hamburg', 'muenchen', "stuttgart", "duesseldorf","koeln", "frankfurt"]
driver = webdriver.Chrome(chrome_options = options)
area_cluster = {}
print('scraping cities')
for city in cities:

    driver.get('https://www.immobilienscout24.de/neubau/{}.html'.format(city))
    time.sleep(3)
    try:
        driver.find_element_by_id('consent_prompt_submit').click()
    except Exception as e:
        continue
    regions = driver.find_elements_by_css_selector("div.section.region--section.grid.gutter.margin-bottom-l")
    results_raw = [tag for tag in regions]
    bezirke = [result.find_element_by_tag_name('h3').text for result in results_raw]
    for index,bezirk in enumerate(bezirke):
        area_cluster[bezirk] = [results_raw[index],index,city]
        
    caption = [result.find_elements_by_css_selector('h3.font-m.font-semibold.font-ellipsis') for result in results_raw]
    flat_caption = [item for sublist in caption for item in sublist if isinstance(sublist,list)]
    caption = [e.text for e in flat_caption]
    
    price = [result.find_elements_by_css_selector('span.absolute-content.padding-vertical-xs.padding-horizontal.font-ellipsis.font-right.font-m.font-semibold.font-white.projectresultlist__price') for result in results_raw]
    flat_price = [item for sublist in price for item in sublist if isinstance(sublist,list)]
    price = [e.text for e in flat_price]
    
    specs = [result.find_elements_by_css_selector('div.grid-item.one-half.font-right.font-ellipsis') for result in results_raw]
    flat_specs = [item for sublist in specs for item in sublist if isinstance(sublist,list)]
    
    area = [e.text for e in flat_specs[0::3]]
    units = [e.text for e in flat_specs[1::3]]
    availability = [e.text for e in flat_specs[2::3]]
    
    href = [result.find_elements_by_css_selector('a') for result in results_raw]
    flat_href = [item for sublist in href for item in sublist if isinstance(sublist,list)]
    href = [e.get_attribute('href') for e in flat_href]
    
    if city == 'berlin':
        df = pd.DataFrame(data = [caption,price,area,units,availability,href]).transpose()
        df['city'] = city
    else:
        df2 = pd.DataFrame(data = [caption,price,area,units,availability,href]).transpose()
        df2['city'] = city
        df = df.append(df2)
    os.system('pkill firefox')
    os.system('pkill chrome')
    os.system('pkill chromium')

df.columns = ['name', 'price', 'area', 'units', 'availability', 'link', 'city']
df['price_min'] = df['price'].apply(lambda x: re.search(r'(.*) - (.*) €',x)[1].replace('.',''))
df['price_max'] = df['price'].apply(lambda x: re.search(r'(.*) - (.*) €',x)[2].replace('.',''))
df.reset_index(inplace = True)
pickle.dump(df, open('cities_df','wb'))

links = df['link'].tolist()
for index, link in enumerate(links):
    progress = index / len(links)
    clear_output(wait=True)
    print('scraping single projects, progress is {}'.format(progress))
    driver = webdriver.Chrome(options=options)
    driver.get(link)
    WebDriverWait(driver,5)
    df['fineprint'] = 'not found'
    df['images'] = 'not found'
    try:
        driver.find_element_by_id('consent_prompt_submit').click()
    except Exception as e:
        continue
    try:
        name = driver.find_elements_by_tag_name("h1")
        name = [e.text for e in name][0]
        df.at[index, 'Project'] = name
    except Exception as e:
        df.at[index, 'Project'] = 'not found'
        project_names.append('')
        continue
    try:
        driver.find_element_by_css_selector('a.icon-arrow.margin-top-xs').click()
    except:
        continue
    try:
        fineprint = driver.find_elements_by_css_selector("p.fineprint")
        fineprint = [e.text for e in fineprint]
        df.at[index, 'fineprint'] = list(fineprint)
    except:
        df.at[index, 'fineprint'] = 'not found'
        continue
    try:
        text = driver.find_elements_by_tag_name('p')
        text= [e.text for e in text if len(e.text) > 90]
        df.at[index,'text'] = ''.join(text)
    except:
        df.at[index,'text'] = 'not found'      
        continue
    try:    
        images = driver.find_elements_by_tag_name('img')
        images = [e.get_attribute('src') for e in images]
        df.at[index,'images'] = ','.join([str(e) for e in images if 'amazonaws' in str(e)]) 
    except:
        df.at[index,'images'] = 'not found'
        continue
    try:
        df.at[index,'promoter'] = driver.find_element_by_css_selector('a.homepage-url').text
    except:
        df.at[index , 'promoter'] = 'not found'
        continue
    try:
        df.at[index,'hp'] = driver.find_element_by_css_selector('a.homepage-url').get_attribute('href')
    except:
        df.at[index,'hp'] = 'not found'
        continue
    try:
        df.at[index, 'contact_name'] = driver.find_element_by_id('detailsContactFooter').find_elements_by_tag_name('p')[0].text
    except:
        df.at[index, 'contact_name'] = 'not found'
        continue
    driver.close()
    driver.quit()
    gc.collect()
    os.system('pkill firefox')
    os.system('pkill chrome')
    os.system('pkill chromium')
    os.system('pkill nightly')
df.to_csv('dump{}.csv'.format(now), encoding = 'utf-8')

scraping single projects, progress is 0.9940828402366864


In [46]:
for index, link in enumerate(links):
    progress = index / len(links)
    clear_output(wait=True)
    print('scraping single projects, progress is {}'.format(progress))
    driver = webdriver.Chrome(options=options)
    driver.get(link)
    WebDriverWait(driver,5)
    df['fineprint'] = 'not found'
    df['images'] = 'not found'
    try:
        driver.find_element_by_id('consent_prompt_submit').click()
    except Exception as e:
        continue
    try:
        name = driver.find_elements_by_tag_name("h1")
        name = [e.text for e in name][0]
        df.at[index, 'Project'] = name
    except Exception as e:
        df.at[index, 'Project'] = 'not found'
        project_names.append('')
        continue
    try:
        driver.find_element_by_css_selector('a.icon-arrow.margin-top-xs').click()
    except:
        continue
    try:
        fineprint = driver.find_elements_by_css_selector("p.fineprint")
        fineprint = [e.text for e in fineprint]
        df.at[index, 'fineprint'] = list(fineprint)
    except:
        df.at[index, 'fineprint'] = 'not found'
        continue
    try:
        text = driver.find_elements_by_tag_name('p')
        text= [e.text for e in text if len(e.text) > 90]
        df.at[index,'text'] = ''.join(text)
    except:
        df.at[index,'text'] = 'not found'      
        continue
    try:    
        images = driver.find_elements_by_tag_name('img')
        images = [e.get_attribute('src') for e in images]
        df.at[index,'images'] = ','.join([str(e) for e in images if 'amazonaws' in str(e)]) 
    except:
        df.at[index,'images'] = 'not found'
        continue
    try:
        df.at[index,'promoter'] = driver.find_element_by_css_selector('a.homepage-url').text
    except:
        df.at[index , 'promoter'] = 'not found'
        continue
    try:
        df.at[index,'hp'] = driver.find_element_by_css_selector('a.homepage-url').get_attribute('href')
    except:
        df.at[index,'hp'] = 'not found'
        continue
    try:
        df.at[index, 'contact_name'] = driver.find_element_by_id('detailsContactFooter').find_elements_by_tag_name('p')[0].text
    except:
        df.at[index, 'contact_name'] = 'not found'
        continue
    driver.close()
    driver.quit()
    gc.collect()
    os.system('pkill firefox')
    os.system('pkill chrome')
    os.system('pkill chromium')
    os.system('pkill nightly')
df.to_csv('dump{}.csv'.format(now), encoding = 'utf-8')

scraping single projects, progress is 0.01775147928994083


KeyboardInterrupt: 

In [50]:
df['link'][0]

'https://www.immobilienscout24.de/neubau/realbest-germany-gmbh/allegria/65063.html'

In [25]:
df_fine

,0,1,2,3,4,5,texts
0,Degnerstraße 20,13053 Berlin,Showroom,29 Wohnungen,01.09.2019,None,"In einem ruhigen Wohngebiet, inmitten idyllisc..."
1,Semmelweisstraße 41-47,12524 Berlin,2. Quartal 2019,None,None,None,Im Bau!\n\nIm Parkquartier Altglienicke finden...
2,Grünauer Straße 57,10245 Berlin,19 Wohnungen,5 Gewerbeeinheiten,Frühsommer 2020,None,Die Baugemeinschaft Wohnen auf dem Lande erric...
3,Möwenweg 21,12683 Berlin,Frühjahr 2019,None,None,None,Berlin boomt. An allen Ecken und Enden entsteh...
4,Nürnberger Straße 68,10789 Berlin,65 Wohnungen,2 Gewerbeeinheiten,Juni 2020,None,Mit dem Projekt „PANDION THE HAUS“ entsteht in...
5,Kasier-Friedrich-Straße 25,10585 Berlin,Showroom,63 Wohnungen,vsl. Mitte 2021,None,Ihr Zuhause in der City West\n\nInmitten Charl...
6,Insterburgallee 21,14055 Berlin,11 Wohnungen,sofort,None,None,3 LETZTE WOHNFÜHL-CHANCEN\n1 Penthouse mit Lof...
7,Schillerstraße 45-46,10627 Berlin,Showroom,80 Wohnungen,1 Gewerbeeinheit,nach Vereinbarung,Das neue 6-geschossige Wohnensemble CARTE BLAN...
8,"Heubnerweg 7-7a, 9-9a",14059 Berlin,Showroom,132 Wohnungen,nach Vereinbarung,None,AM SCHLOSSPARK steht für Vielfalt: Vom kompakt...
9,Englische Straße 20,10587 Berlin,nach Vereinbarung,None,None,None,Zentral residieren und geradezu idyllisch wohn...


In [26]:
project_names

['Allegria',
 'Parkquartier Altglienicke',
 'Wohnen auf dem Lande',
 "Stadtvillen an der Wuhle 'Grüne Aue'",
 'PANDION THE HAUS',
 'Kaiser im Kiez',
 'Insterburgallee 21',
 'CARTE BLANCHE',
 'AM SCHLOSSPARK',
 'No.1 Charlottenburg',
 'WESTENDPARK pure',
 'fiftysix',
 '* Über 60% verkauft * Spandauer54',
 'WIELAND41',
 'Spielhagenstraße 8',
 'L85',
 '54 Urban Penthouses Charlottenburg',
 'Fürstenplatz',
 'Aedes Nobiles',
 'MES5',
 'Luise11',
 'Gutshof Falkenberg',
 'Gutshof Falkenberg',
 'Berliner Straße 46-49',
 'Vive la Rose',
 'PRE SALE French Garden',
 'Berliner Straße 46-49',
 'Maison VIKTORIA',
 'I LOVE Lichtenberg',
 'FUCHSBAU Lichtenberg',
 'Lückstraße 38',
 'Neubau in Gartenlage nahe (Media) Spree',
 'BOX SEVEN',
 'Jessner Höfe - Top-Lage nahe Traveplatz',
 'PURE',
 'Kopernikusstr. 30 | Warschauer Str. 65',
 'UPSIDE BERLIN - MIT WEITBLICK WOHNEN',
 'WAVE WATERSIDE LIVING BERLIN',
 'SCHREIBFEDERHÖFE',
 'Revaler Spitze',
 'UPSIDE BERLIN - MIT WEITBLICK WOHNEN',
 'Havelgärten Hohe

In [27]:
fineprints

[['Degnerstraße 20', '13053 Berlin', 'Showroom', '29 Wohnungen', '01.09.2019'],
 ['Semmelweisstraße 41-47', '12524 Berlin', '2. Quartal 2019'],
 ['Grünauer Straße 57',
  '10245 Berlin',
  '19 Wohnungen',
  '5 Gewerbeeinheiten',
  'Frühsommer 2020'],
 ['Möwenweg 21', '12683 Berlin', 'Frühjahr 2019'],
 ['Nürnberger Straße 68',
  '10789 Berlin',
  '65 Wohnungen',
  '2 Gewerbeeinheiten',
  'Juni 2020'],
 ['Kasier-Friedrich-Straße 25',
  '10585 Berlin',
  'Showroom',
  '63 Wohnungen',
  'vsl. Mitte 2021'],
 ['Insterburgallee 21', '14055 Berlin', '11 Wohnungen', 'sofort'],
 ['Schillerstraße 45-46',
  '10627 Berlin',
  'Showroom',
  '80 Wohnungen',
  '1 Gewerbeeinheit',
  'nach Vereinbarung'],
 ['Heubnerweg 7-7a, 9-9a',
  '14059 Berlin',
  'Showroom',
  '132 Wohnungen',
  'nach Vereinbarung'],
 ['Englische Straße 20', '10587 Berlin', 'nach Vereinbarung'],
 ['Tharauer Allee 4-12', '14055 Berlin', 'Bezugsfertig'],
 ['Lietzenburger Straße 56',
  '10719 Berlin',
  '2 Wohnungen',
  '49 Gewerbeeinh

In [23]:
len(texts)

458

In [ ]:
df = pickle.load(open('cities_df','rb'))
links = df['link'].tolist()

project_names = []
fineprints = []
texts = []
images_set = []
promoters = []
promoter_hp = []
contact_names = []
phone_contact = []

for index, link in enumerate(links):
    progress = index / len(links)
    clear_output(wait=True)
    print(f'scraping single projects, progress is {progress}')
    driver = webdriver.Firefox(options=options)

    driver.get(link)
    try:
        driver.find_element_by_id('consent_prompt_submit').click()
    except Exception as e:
        continue
    try:
        name = driver.find_elements_by_tag_name("h1")
        name = [e.text for e in name][0]
        project_names.append(name)
    except Exception as e:
        project_names.append('')
        continue
    try:
        driver.find_element_by_css_selector('a.icon-arrow.margin-top-xs').click()
    except:
        continue
    try:
        fineprint = driver.find_elements_by_css_selector("p.fineprint")
        fineprint = [e.text for e in fineprint]
        fineprints.append(fineprint)
    except:
        fineprints.append('')
        continue
    try:
        text = driver.find_elements_by_tag_name('p')
        text= [e.text for e in text if len(e.text) > 90]
        texts.append(''.join(text))
    except:
        texts.append('')
        continue
    try:    
        images = driver.find_elements_by_tag_name('img')
        images = [e.get_attribute('src') for e in images]
        images = [str(e) for e in images if 'amazonaws' in str(e)]
        images_set.append(images)
    except:
        images.append('')
        continue
    try:
        promoters.append(driver.find_element_by_css_selector('a.homepage-url').text)
        promoter_hp.append(driver.find_element_by_css_selector('a.homepage-url').get_attribute('href'))
        contact_names.append(driver.find_element_by_id('detailsContactFooter').find_elements_by_tag_name('p')[0].text)
    except:
        promoters.append('')
        promoter_hp.append('')
        contact_names.append('')
        continue
    driver.close()
    driver.quit()
    print(fineprint)
    gc.collect()
df['project_names'] = project_names
df['fineprints'] = fineprints
df['texts'] = texts
df['images_set'] = images_set
df['promoters'] = promoters
df['promoter_hp'] = promoter_hp
df['contact_names'] = contact_names
df['phone_contact'] = phone_contact

scraping single projects, progress is 0.15294117647058825


In [6]:
driver.close()

In [81]:
df['price_min'] = df['price'].apply(lambda x: re.search(r'(.*) - (.*) €',x)[1].replace('.',''))
df['price_max'] = df['price'].apply(lambda x: re.search(r'(.*) - (.*) €',x)[2].replace('.',''))
df.reset_index(inplace = True)

In [359]:
len(project_names) == len(fineprints) == len(contact_names)

True

In [357]:
contact_names

['Jens Schulte',
 'Ihr Team realbest Germany GmbH',
 'Niederlassung 5 Berlin',
 'Carl Oesterley',
 'Erik-Martin Zöbisch',
 'Ihr WvM Berlin Vertrieb',
 'Jürgen Meier-Loser',
 'NEUBAUPROJEKTE IN BESTEN LAGEN',
 'Ihr Makler in Berlin!',
 '',
 'Tina Jochmann']

In [85]:
print(df.shape)
df.drop_duplicates(inplace = True)
print(df.shape)

(514, 10)
(514, 10)


### Playground

### Single Project

In [86]:
df.head()

,index,name,price,area,units,availability,link,city,price_min,price_max
0,0,Attraktive Kapitalanlage in Adlershof,144.500 - 237.000 €,"29,00 - 46,00 m²",25,vermietet,https://www.immobilienscout24.de/neubau/euv-pr...,berlin,144500,237000
1,1,Wohnfreude hoch Drei,243.334 - 605.570 €,"47,87 - 133,68 m²",29,01.09.2019,https://www.immobilienscout24.de/neubau/realbe...,berlin,243334,605570
2,2,Modern ausgestattete Eigentumswohnungen und Re...,121.000 - 566.000 €,"31,00 - 134,00 m²",80,2. Quartal 2019,https://www.immobilienscout24.de/neubau/projec...,berlin,121000,566000
3,3,Baugemeinschaft Grünauer Straße GbR,342.600 - 657.000 €,"81,00 - 148,00 m²",24,Frühsommer 2020,https://www.immobilienscout24.de/neubau/gbr-gr...,berlin,342600,657000
4,4,"Stadtvillen an der Wuhle ""Grüne Aue""",239.500 - 262.000 €,"58,00 - 64,00 m²",36,Frühjahr 2019,https://www.immobilienscout24.de/neubau/verima...,berlin,239500,262000


In [88]:
links = df['link'].tolist()

In [337]:
project_names = []
fineprints = []
texts = []
images_set = []
promoters = []
promoter_hp = []
contact_names = []
phone_contact = []
links_test = links[1:4]
print(links_test)

for index, link in enumerate(links):
    driver = webdriver.Firefox()

    driver.get(link)
    time.sleep(2)
    try:
        driver.find_element_by_id('consent_prompt_submit').click()
    except Exception as e:
        continue
    try:
        name = driver.find_elements_by_tag_name("h1")
        name = [e.text for e in name][0]
        project_names.append(name)
    except Exception as e:
        continue
    try:
        driver.find_element_by_css_selector('a.icon-arrow.margin-top-xs').click()
    except:
        continue
    try:
        fineprint = driver.find_elements_by_css_selector("p.fineprint")
        fineprint = [e.text for e in fineprint]
        fineprints.append(fineprint)
    except:
        continue
    try:
        text = driver.find_elements_by_tag_name('p')
        text= [e.text for e in text if len(e.text) > 90]
        texts.append(''.join(text))
    except:
        continue
    try:    
        images = driver.find_elements_by_tag_name('img')
        images = [e.get_attribute('src') for e in images]
        images = [str(e) for e in images if 'amazonaws' in str(e)]
        images_set.append(images)
    except:
        continue
    try:
        promoters.append(driver.find_element_by_css_selector('a.homepage-url').text)
        promoter_hp.append(driver.find_element_by_css_selector('a.homepage-url').get_attribute('href'))
        contact_names.append(driver.find_element_by_id('detailsContactFooter').find_elements_by_tag_name('p')[0].text)
    except:
        continue
    driver.quit()
df['project_names'] = project_names
df['fineprints'] = fineprints
df['texts'] = texts
df['images_set'] = images_set
df['promoters'] = promoters
df['promoter_hp'] = promoter_hp
df['contact_names'] = contact_names
df['phone_contact'] = phone_contact

['https://www.immobilienscout24.de/neubau/realbest-germany-gmbh/allegria/65063.html', 'https://www.immobilienscout24.de/neubau/project-immobilien/parkquartier-altglienicke/56552.html', 'https://www.immobilienscout24.de/neubau/gbr-gruenauer-strasse/wohnen-auf-dem-lande/76396.html']


In [341]:
fineprints

[['Degnerstraße 20', '13053 Berlin', 'Showroom', '29 Wohnungen', '01.09.2019'],
 ['Semmelweisstraße 41-47', '12524 Berlin', '2. Quartal 2019'],
 ['Grünauer Straße 57',
  '10245 Berlin',
  '19 Wohnungen',
  '5 Gewerbeeinheiten',
  'Frühsommer 2020']]

In [288]:
df.head()

,index,name,price,area,units,availability,link,city,price_min,price_max
0,0,Attraktive Kapitalanlage in Adlershof,144.500 - 237.000 €,"29,00 - 46,00 m²",25,vermietet,https://www.immobilienscout24.de/neubau/euv-pr...,berlin,144500,237000
1,1,Wohnfreude hoch Drei,243.334 - 605.570 €,"47,87 - 133,68 m²",29,01.09.2019,https://www.immobilienscout24.de/neubau/realbe...,berlin,243334,605570
2,2,Modern ausgestattete Eigentumswohnungen und Re...,121.000 - 566.000 €,"31,00 - 134,00 m²",80,2. Quartal 2019,https://www.immobilienscout24.de/neubau/projec...,berlin,121000,566000
3,3,Baugemeinschaft Grünauer Straße GbR,342.600 - 657.000 €,"81,00 - 148,00 m²",24,Frühsommer 2020,https://www.immobilienscout24.de/neubau/gbr-gr...,berlin,342600,657000
4,4,"Stadtvillen an der Wuhle ""Grüne Aue""",239.500 - 262.000 €,"58,00 - 64,00 m²",36,Frühjahr 2019,https://www.immobilienscout24.de/neubau/verima...,berlin,239500,262000


In [296]:
project_names

[['Attraktive Kapitalanlage in Adlershof']]

In [298]:
contact_names

['Jens Schulte']

In [299]:
project_names

[['Attraktive Kapitalanlage in Adlershof']]

In [300]:
post_codes

[['12489 Berlin']]

In [301]:
promoters

['EuV Projekte GmbH']

In [302]:
images_set

[['https://pictures.immobilienscout24.de/dims3/S3/quality/80/resize/1170x600%3E/http://s3-eu-west-1.amazonaws.com/pda-pro-pictures-projectpictures-8hecgpgpb9fo/67783493/bccf2ff0-cee5-477b-824f-e12fcb114ef1.jpg',
  'https://pictures.immobilienscout24.de/dims3/S3/quality/80/resize/1170x600%3E/http://s3-eu-west-1.amazonaws.com/pda-pro-pictures-projectpictures-8hecgpgpb9fo/67783493/a34fdbbb-fcc9-4c39-bb42-4277aa54742e.jpg',
  'https://pictures.immobilienscout24.de/dims3/S3/quality/80/resize/1170x600%3E/http://s3-eu-west-1.amazonaws.com/pda-pro-pictures-projectpictures-8hecgpgpb9fo/67783493/352c1415-5a2a-47a7-99f4-5d5fae20eb15.jpg',
  'https://pictures.immobilienscout24.de/dims3/S3/quality/80/resize/1170x600%3E/http://s3-eu-west-1.amazonaws.com/pda-pro-pictures-projectpictures-8hecgpgpb9fo/67783493/bccf2ff0-cee5-477b-824f-e12fcb114ef1.jpg']]

'https://www.immobilienscout24.de/neubau/realbest-germany-gmbh/allegria/65063.html'

In [306]:
project_names

[['Attraktive Kapitalanlage in Adlershof']]

In [307]:
fineprint

['Dörpfeldstraße 5', '12489 Berlin', '25 Wohnungen', 'vermietet']

In [310]:
project_names

[['Wohnfreude hoch Drei']]

In [311]:
addresses

[['Degnerstraße 20', '01.09.2019']]

In [313]:
fineprints

[['Degnerstraße 20', '13053 Berlin', 'Showroom', '29 Wohnungen', '01.09.2019']]

In [314]:
texts

['In einem ruhigen Wohngebiet, inmitten idyllischer Umgebung mit teilweise historischen Bauten, entsteht ein modernes Gebäudeensemble mit qualitativ hochwertigem Anspruch. Insgesamt werden hier 29 Wohneinheiten in zwei Vorderhäusern und einem Gartenhaus entstehen. Vom Erdgeschoss bis zur 5. Etage werden 2- bis 4-Zimmer-Wohnungen mit Wohnflächen von rund 47 m² bis zu 133 m² angeboten. Der Großteil der Wohnungen wird über Terrassen oder Balkone bzw. Loggien verfügen. Vom Untergeschoss bis zum Dachgeschoss werden Sie alle Etagen bequem mit einem Aufzug erreichen können. Im Untergeschoss wird zu jeder Wohneinheit ein abschließbarer Kellerraum zur Verfügung stehen. Eine zentrale Schließanlage für Hauseingangstür, Kellerzugang und Wohnungseingang erspart Ihnen das übliche Schlüsselchaos. Den Erdgeschosswohnungen ist ein Privatgarten zugeordnet, der von den neuen Eigentümern selbst gestaltet werden kann. Die restliche Außenanlage wird als Gemeinschaftseigentum liebevoll mit eleganten Leuchtmi

In [317]:
promoter_hp

['https://www.realbest.de/']

In [318]:
contact_names

['Ihr Team realbest Germany GmbH']

In [319]:
phone_contact

[]

In [321]:
fineprints

[['Dörpfeldstraße 5', '12489 Berlin', '25 Wohnungen', 'vermietet']]

In [322]:
promoter_hp

['https://www.engelvoelkers.com/de-de/berlin/projekte/']

In [1]:
import pickle

In [ ]:
pickle.dump()